In [ ]:
!pip install livelossplot
from livelossplot import PlotLossesKeras
plotlosses = PlotLossesKeras()

In [ ]:
# 1.Import libraries
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams


from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import keras

from keras.utils import np_utils

from tensorflow.keras import backend as K

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization,Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

print("Tensorflow version:", tf.__version__)


In [ ]:
#Set figure size
rcParams['figure.figsize'] = 20, 10

In [ ]:
2. #Import the dataset
# data_path = '../input/effaj-dataset2/JAFFE_recompiled'
data_path = '../input/effaj-augmentation/JAFFE_recompiled'
data_dir_list = sorted(os.listdir(data_path))
data_dir_list

In [ ]:
img_rows=256
img_cols=256
num_channel=1

num_epoch=10

img_data_list=[]


for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print (f'Loaded the images of dataset {dataset}: {len(img_list)} images\n')
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
#         print(len(img_list))
        #input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize)
        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

In [ ]:
#3.Define the number of classes and attach class number
num_classes = 7

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:209]=0 #30 -> ANGRY
labels[210:410]=1 #29 -> DISGUSY
labels[411:633]=2 #32 -> FEAR
labels[634:850]=3 #31 -> NEUTRAL
labels[851:1059]=4 #30 -> HAPPY
labels[1060:1274]=5 #31 -> SAD
labels[1275:]=6 #30 -> SURPRISE

names = ['ANGRY','DISGUST','FEAR','HAPPY','NEUTRAL','SAD','SURPRISE']

def getLabel(id):
    return ['ANGRY','DISGUST','FEAR','HAPPY','NEUTRAL','SAD','SURPRISE'][id]

In [ ]:
#4.Convert the class label to one-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=2)
x_test=X_test
print(f"Train:{X_train.shape}\nTest:{X_test.shape}")

In [ ]:
# 5. Build CNN layer
# Defining the model

input_shape=(128,128,3)

model = Sequential()

model.add(Conv2D(6, (5, 5), input_shape=input_shape, padding='same', activation = 'relu'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (5, 5), padding='same', activation = 'relu'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(120, (5, 5), activation = 'relu'))
model.add(Activation('relu'))
model.add(Dropout(0.3))



model.add(Flatten())
model.add(Dense(84, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation = 'softmax'))


# Classification
# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

#Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])

model.summary()


In [ ]:
#5. Train model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau , EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor = 0.01 , patience=10 , 
                             min_lr=0.00001 , mode='auto')
earlystop = EarlyStopping(monitor ='val_loss',
                         min_delta = 0, #abs value criteria for stopping
                         patience = 2, #no of epochs to wait before stopping
                         verbose = 1,
                         restore_best_weights=True #keep the best weights once stopped
                         )
hist = model.fit(X_train, y_train, batch_size=7, epochs=50, verbose=1, validation_data=(X_test, y_test),callbacks=[reduce_lr])